In [11]:
import re

import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from pypdl import Downloader
from urllib.parse import parse_qsl

In [12]:
play_url = "https://play.google.com/store/apps/details?id=com.zhiliaoapp.musically"

In [13]:
d = Downloader()
driver = Chrome()
driver.maximize_window()

In [14]:
def get_package_name(google_play_url):
    # example url https://play.google.com/store/apps/details?id=com.google.android.apps.photosgo
    return parse_qsl(google_play_url)[0][1]

In [20]:
def get_screenshots(driver, package_name):
    url = f"https://apkcombo.com/search?q={package_name}"
    driver.get(url)
    
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try:
        app_name = soup.find("div", {"class": "app_name"}).text
        urls = []
        for url_soup in soup.find("div", {"id": "gallery-screenshots"}).find_all("a"):
            urls.append(url_soup.get("href"))
            
        download_url = 
        return app_name, urls
    except AttributeError:
        print("No app name found")
        return False

In [21]:
get_screenshots(driver, "com.google.android.apps.photosgo")

('Gallery',
 ['https://play-lh.googleusercontent.com/IvL1Y_oDYHAdKvPEUpwk4NbIFdtMtOqltmgh10Y5DjmtefHwFoXgKzw8HZZVzjqU1fk=w1024-h768-rw',
  'https://play-lh.googleusercontent.com/awhwVd1gWUx-7IgFq1ofBa5kTYmTwC-X6o9yXsXV-Uer-K-jDPG_F-VEkl1gNE0dVps=w1024-h768-rw',
  'https://play-lh.googleusercontent.com/Jru7R3BDQlrjpfOJGvDAxxeXujB0B4M7ltrfslwlh2VCwSBicPbr5j5Cqyoiy0Zs-6w=w1024-h768-rw',
  'https://play-lh.googleusercontent.com/C9w6JlYk1wRSBrNM6VPe0y0cYg_ZXlLccI5kAfNH9qzBcazCLbrFUMJt3DphDO2LNg8=w1024-h768-rw',
  'https://play-lh.googleusercontent.com/X5vP9HmTwiEL2i0f9R_olVDPlrEaa00jS_vUn1dhx3cCz_rjU633yu7--xw8ZUpwzSlT=w1024-h768-rw'])

In [22]:
play_urls = """https://play.google.com/store/apps/details?id=com.toka.town.life.world.avatar.toca.boca.miga.toga.tira.my.world
https://play.google.com/store/apps/details?id=com.Avatar.World.City.Life.tira.town.miga.toca.boca.Avitar
https://play.google.com/store/apps/details?id=com.toka.town.life.world.avatar.toca.boca.miga.toga.tira.my.world.castle
https://play.google.com/store/apps/details?id=com.anime.sakura.school.simulator.life.love.games
https://play.google.com/store/apps/details?id=com.stair.run.bridge.race.bridgeway.game
https://play.google.com/store/apps/details?id=com.diy.bubble.tea.tapioca.recipe.tasty
https://play.google.com/store/apps/details?id=com.antistress.relaxing.games.stress.puppet
https://play.google.com/store/apps/details?id=com.adnime.vlinder.princesss.dress.up.fashion.everskies.girl.games
https://play.google.com/store/apps/details?id=com.love.nikki.adnime.dress.up.simulator.games
https://play.google.com/store/apps/details?id=com.happy.township.farming.free.games
https://play.google.com/store/apps/details?id=com.toka.hair.salon.makeup.games
https://play.google.com/store/apps/details?id=com.ldle.market.tycoon.shopcooking
https://play.google.com/store/apps/details?id=com.pranks.blastet.neaf.epic.game
https://play.google.com/store/apps/details?id=com.anime.fashion.princess.girl.dress.up
https://play.google.com/store/apps/details?id=com.world.war.mission.strikes.offline.game""".split("\n")

In [26]:
data = {}
for i in play_urls:
    package_name = get_package_name(i)
    screenshots = get_screenshots(driver, package_name)
    if not screenshots:
        continue
    data[screenshots[0]] = screenshots[1]

No app name found


In [48]:
def get_play_screenshots(play_url):
    response = requests.get(play_url)
    soup = BeautifulSoup(response.text, "html.parser")
    try:
        name = soup.find("h1", {"itemprop": "name"}).text
        urls = dict()
        for url_soup in soup.find("div", {"jsname": "K9a4Re"}).find_all("img"):
            urls[f"Screenshot {len(urls)}.png"] = url_soup.get("src").split("=")[0]
        return name, urls
    except AttributeError:
        return False

In [43]:
data = {}
for i in play_urls:
    # package_name = get_package_name(i)
    screenshots = get_play_screenshots(i)
    if not screenshots:
        continue
    data[screenshots[0]] = screenshots[1]

In [45]:
import os
from requests.exceptions import ChunkedEncodingError, ConnectionError


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/89.0.4389.114 Safari/537.36",
    "referer": 'https://apkcombo.com/'
}

def download_n_save(url, filename, save_path_dir, retry=0):
    # logger.debug(f"downloading {url}")
    print("downloading {} to {}".format(url, filename))
    filepath = os.path.join(save_path_dir, filename)

    with requests.get(url, stream=True, allow_redirects=True, headers=headers) as req:
        if not req.ok:
            # logger.error(f"unable to download {filename} file..")
            req.raise_for_status()
        try:
            with open(filepath, "wb") as fp:
                for data in req.iter_content(chunk_size=8192):
                    fp.write(data)
        except ChunkedEncodingError as e:
            if retry > 3:
                os.remove(filepath)
                # logger.debug(f"unable to download from {url} after retrying 3 times.")
                return ""
            # logger.debug(f"Failed to get data from {url}, retrying {retry+1} time")
            download_n_save(url, filename, save_path_dir, retry=retry+1)
    return filepath


In [47]:
for apk_name in data:
    print(apk_name)
    for i, url in enumerate(data[apk_name]):
        download_n_save(url, f'screenshots {apk_name} {i}.png', r"C:\Users\Anon\Downloads\a")

MY Town Life World
downloading https://play-lh.googleusercontent.com/6dmnRhY-IbHPeT201KqkLyUoxgIGkf-4jIuGpg_MAcVVk_fOxLwbcwXrF801LVkRDWM=w526-h296 to screenshots MY Town Life World 0.png
downloading https://play-lh.googleusercontent.com/igxqHXnkE5yG2IgY_vfq_eitBTOQ0v7PaP4TTYgmipZGOVgxN0HNHwWy7dt0HocT2lc=w526-h296 to screenshots MY Town Life World 1.png
downloading https://play-lh.googleusercontent.com/OdXmyDSW_Pwb9Bk2A5zzzPHqYjkahaE6mEXDY6Ut2fTOEMKukfAnnsFed5G11GEwbw=w526-h296 to screenshots MY Town Life World 2.png
downloading https://play-lh.googleusercontent.com/EDOeVLTTZQY9VB3p_TDavc-dIaGAshusqBz8tyqnCiSGZhgmR9HwJmV5u47b-02VOg=w526-h296 to screenshots MY Town Life World 3.png
downloading https://play-lh.googleusercontent.com/ECDuwA_9yVOGS4HYFiZCYsiMAwKJLLPYFKwWMvkvNcUELuOzHnfaaHwD3Mo5YTrMqKE=w526-h296 to screenshots MY Town Life World 4.png
downloading https://play-lh.googleusercontent.com/XdgGQlMWw_-HpH4MAejMYSeiPzKjB3hEDKjG3VxLeItJu6e6b6hBPmLtIySsY3NTjA=w526-h296 to screenshots M

KeyboardInterrupt: 

In [46]:
data.keys()

dict_keys(['MY Town Life World', 'MY Town Life World Castle', 'High Girl school simulator 3D', 'Stair Run Bridge Race 3D', 'Boba Drink Tasty Bubble Tea', 'Antistress - relaxing games', 'Anime vinder princess dress up', 'Dress up Anime Star Princess', 'Happy township farming games', 'Hair Braided Salon Makeup Game', 'Supermarket simulator:my store', 'John Trigger 3d:Action Shooter', 'Makeup princess anime dress up', 'FPS war Strike:PVP Shooter'])

In [52]:
a = d.start("https://download.apkcombo.com/com.microsoft.outlooklite/Microsoft%20Outlook%20Lite:%20Email_3.24.0-minApi24_apkcombo.com.apk?ecp=Y29tLm1pY3Jvc29mdC5vdXRsb29rbGl0ZS8zLjI0LjAtbWluQXBpMjQvNjAzMjQwLjU2YzE0MDhkZGMxNmIyZjUzMjk3OGY4ODVkMTdjMGE0YWIyNzc4MjMuYXBr&iat=1707057013&sig=baa29b8e3b7282f870fdc093483e331b&size=14522564&from=cf&version=latest&lang=en&fp=1eaa3ef0653ef76edc7cc5e4161b6c00&ip=104.28.220.192", file_path=os.path.join(os.getcwd(), "abc.apk"), retries=3, segments=5, display=False)

a

Not in terminal, reprint now using normal build-in print function.


In [56]:
import time
from concurrent.futures import ThreadPoolExecutor

import requests
from bs4 import BeautifulSoup
from urllib.parse import parse_qsl, urlunparse, urlparse, urlencode
from pathlib import Path
import os
# from logger import logger
from PIL import Image
from selenium.webdriver import Chrome, ChromeOptions
from requests.exceptions import ChunkedEncodingError, ConnectionError
from pypdl import Downloader


In [57]:
WRK_DIR = os.getcwd()
APK_DATA_PATH = os.path.join(WRK_DIR, "src", "apk_data")
py_downloader = Downloader()
chrome_options = ChromeOptions()
chrome_options.add_argument("--headless")
driver = Chrome()
driver.maximize_window()

In [ ]:
url = "https://apkcombo.com/autokill-sandbox/{package_name}/download/apk"
driver.get(url.format(package_name=package_name))
time.sleep(5)
soup = BeautifulSoup(driver.page_source, "html.parser")
download_link = soup.find("a", class_="variant").get("href")
driver.close()
logger.debug(f"Extracted download url from apkcombo {download_link}")

In [64]:


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/89.0.4389.114 Safari/537.36",
    "referer": 'https://apkcombo.com/'
}

a = requests.get("https://apkcombo.com/search?q=com.microsoft.outlooklite", headers=headers)

In [66]:
pattern = r'/([^/]+)/$'
url = f"https://apkcombo.com/search?q=com.microsoft.outlooklite"

driver.get(url)
match = re.search(pattern, driver.current_url)
if match:
    middle_apk_name = match.group(0)

[<selenium.webdriver.remote.webelement.WebElement (session="a604a8d54112083216bea023db11e635", element="556A795F99D91922F2BE627BBD687274_element_25")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a604a8d54112083216bea023db11e635", element="556A795F99D91922F2BE627BBD687274_element_109")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a604a8d54112083216bea023db11e635", element="556A795F99D91922F2BE627BBD687274_element_1143")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a604a8d54112083216bea023db11e635", element="556A795F99D91922F2BE627BBD687274_element_1147")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a604a8d54112083216bea023db11e635", element="556A795F99D91922F2BE627BBD687274_element_1151")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a604a8d54112083216bea023db11e635", element="556A795F99D91922F2BE627BBD687274_element_119")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a604a8d54112083216be

In [69]:
re.findall(driver.current_url)

'https://apkcombo.com/microsoft-outlook-lite-email/com.microsoft.outlooklite/'

In [77]:
import re
url = 'https://apkcombo.com/microsoft-outlook-lite-email/com.microsoft.outlooklite/'

# Define a regular expression pattern to extract the desired part
pattern = r'/([^/]+)/$'

# Use re.search to find the pattern in the URL
match = re.search(pattern, url)
match.group(0)

'/com.microsoft.outlooklite/'